In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import re

In [2]:
# Read in the data as a pandas DataFrame, with encoding attribute set to properly read in the data
df = pd.read_csv('WELLCOME_APCspend2013_forThinkful.csv', encoding='latin-1')

df.head(10) # Viewing top of dataframe to get a sense of row/column info

,PMID/PMCID,Publisher,Journal title,Article title,COST (£) charged to Wellcome (inc VAT when charged)
0,NaN,CUP,Psychological Medicine,Reduced parahippocampal cortical thickness in ...,£0.00
1,PMC3679557,ACS,Biomacromolecules,Structural characterization of a Model Gram-ne...,£2381.04
2,23043264 PMC3506128,ACS,J Med Chem,"Fumaroylamino-4,5-epoxymorphinans and related ...",£642.56
3,23438330 PMC3646402,ACS,J Med Chem,Orvinols with mixed kappa/mu opioid receptor a...,£669.64
4,23438216 PMC3601604,ACS,J Org Chem,Regioselective opening of myo-inositol orthoes...,£685.88
5,PMC3579457,ACS,Journal of Medicinal Chemistry,Comparative Structural and Functional Studies ...,£2392.20
6,PMC3709265,ACS,Journal of Proteome Research,Mapping Proteolytic Processing in the Secretom...,£2367.95
7,23057412 PMC3495574,ACS,Mol Pharm,Quantitative silencing of EGFP reporter gene b...,£649.33
8,PMCID: PMC3780468,ACS (Amercian Chemical Society) Publications,ACS Chemical Biology,A Novel Allosteric Inhibitor of the Uridine Di...,£1294.59
9,PMCID: PMC3621575,ACS (Amercian Chemical Society) Publications,ACS Chemical Biology,Chemical proteomic analysis reveals the drugab...,£1294.78


### My first question to answer is: What are the 5 most common journals and the total articles for each?

In [3]:
# To get a better count for our journal titles, convert all to lower case to remove any Capitalization differences that might throw off our counts
df['Journal title'] = df['Journal title'].str.lower()

# Find the Top 5 Journals based on number of articles in the dataset
df['Journal title'].value_counts().head(5)

plos one                           190
journal of biological chemistry     53
neuroimage                          29
plos genetics                       24
plos pathogens                      24
Name: Journal title, dtype: int64

In [4]:
# To ensure these counts are as accurate as possible, let's try and get rid of any duplicate or missing 
# articles that appear in the data.
# I'll start by extracting the PMCID and PMID data to find any matching id's and remove them from the set

# Creating a function that obtains the 7-digit id that starts with 3 from the alphanumeric id which is the PMCID
def get_pmcid(x):
    pmcid = re.findall('3\d{6}', str(x))
    if len(pmcid) > 0:
        return pmcid[- 1]

# Next function obtains the 8-digit id that starts with 2 from the alphanumeric id col, which is the PMID
def get_pmid(x):
    pmid = re.findall('2\d{7}', str(x))
    if len(pmid) > 0:
        return pmid[0]

# Applying our functions to the PMID/PMCID col, and creating two new cols
df['PMID'] = df['PMID/PMCID'].apply(get_pmid)
df['PMCID'] = df['PMID/PMCID'].apply(get_pmcid)

# Now dropping the old column
del df['PMID/PMCID']

# With our new columns in place, we can drop any duplicate ids that exist:
df.drop_duplicates(subset='PMID', keep="last", inplace=True)
df.drop_duplicates(subset='PMCID', keep="last", inplace=True)

# Dropping rows with no IDs
df.drop(df[(df.PMID == None) & (df.PMCID == None)].index, inplace=True)

In [5]:
# After cleaning, let's check our article counts per journal
df['Journal title'].value_counts().head(5)

plos one                           23
journal of general virology         8
journal of biological chemistry     7
elsevier                            7
plos pathogens                      5
Name: Journal title, dtype: int64

### To our 2nd question: What are the mean, median, and standard deviation for the cost of an article per Journal?

In [6]:
# Immediately notice complicated column title for COST, let's simplify
df['cost'] = df['COST (£) charged to Wellcome (inc VAT when charged)']

In [7]:
# Deleting the old column
del df['COST (£) charged to Wellcome (inc VAT when charged)']

In [8]:
# There are some unnecessary currency symbols in the 'cost' column data, clean for later statistical analysis and manipulation
df['cost'] = df.cost.str.replace('£', '')
df['cost'] = df.cost.str.replace('$', '')

df.head()

,Publisher,Journal title,Article title,PMID,PMCID,cost
2,ACS,j med chem,"Fumaroylamino-4,5-epoxymorphinans and related ...",23043264,3506128,642.56
3,ACS,j med chem,Orvinols with mixed kappa/mu opioid receptor a...,23438330,3646402,669.64
4,ACS,j org chem,Regioselective opening of myo-inositol orthoes...,23438216,3601604,685.88
7,ACS,mol pharm,Quantitative silencing of EGFP reporter gene b...,23057412,3495574,649.33
18,American Association of Immunologists,journal of immunology,Regulation of Foxp3+ Inducible Regulatory T Ce...,23455506,3607399,3108.08


In [9]:
# Checking datatype of cost column, may require reformatting
df.cost.dtype

dtype('O')

In [10]:
# Convert data to proper format for statistical analysis
df['cost'] = df['cost'].astype('float64')

In [11]:
# Want to get rid of any empty cost values
df.dropna(subset=['cost'], inplace=True)

df.head(10)

,Publisher,Journal title,Article title,PMID,PMCID,cost
2,ACS,j med chem,"Fumaroylamino-4,5-epoxymorphinans and related ...",23043264,3506128,642.56
3,ACS,j med chem,Orvinols with mixed kappa/mu opioid receptor a...,23438330,3646402,669.64
4,ACS,j org chem,Regioselective opening of myo-inositol orthoes...,23438216,3601604,685.88
7,ACS,mol pharm,Quantitative silencing of EGFP reporter gene b...,23057412,3495574,649.33
18,American Association of Immunologists,journal of immunology,Regulation of Foxp3+ Inducible Regulatory T Ce...,23455506,3607399,3108.08
19,American Chemical Society,acs chemical biology,Discovery of an allosteric inhibitor binding s...,24015914,3833349,1267.76
24,American Chemical Society,american chemical society,Characterizing the Fatty Acid Binding Site in ...,22971149,3466778,1533.29
25,American Chemical Society,analytical chemistry,Amphipathic polymers facilitate the study of f...,23072351,3072351,2082.74
28,American Chemical Society,bioconjugate chemistry,"Design, synthesis and functional activity of l...",23458425,3630740,1565.26
31,American Chemical Society,journal of medicinal chemistry,A class of 5-nitro-2-furancarboxylamides with ...,23281892,3579312,321.12


In [13]:
# remove NA values or any obviously wrong price:
df.cost.fillna(0, inplace=True)
df.drop(df[(df.cost > 10000.00)].index, inplace=True)

In [19]:
# With a cleaned 'cost' col, let's answer our second question by getting the mean, median, and std.deviation for each journal
df.groupby('Journal title').cost.describe().sort_values(by='count', ascending=False)

,count,mean,std,min,25%,50%,75%,max
Journal title,,,,,,,,
plos one,22.0,1056.374091,252.506052,747.05,886.9500,1037.825,1072.9075,1745.00
journal of general virology,8.0,2034.375000,185.615530,1575.00,2100.0000,2100.000,2100.0000,2100.00
journal of biological chemistry,7.0,1654.457143,565.026036,1119.61,1143.6100,1609.660,2031.8200,2501.07
elsevier,7.0,2625.421429,964.947531,2152.76,2186.4400,2269.150,2391.5800,4800.00
plos pathogens,5.0,1560.492000,178.054111,1397.00,1440.0000,1460.300,1723.1600,1782.00
nucleic acids research,5.0,1629.600000,481.826525,852.00,1704.0000,1704.000,1704.0000,2184.00
journal of neuroscience,4.0,1751.795000,343.061020,1283.00,1618.9925,1823.455,1956.2575,2077.27
chembiochem,4.0,2034.242500,129.568271,1851.15,1999.4850,2066.205,2100.9625,2153.41
neuroimage,4.0,2192.320000,292.179787,1758.89,2149.8275,2312.570,2355.0625,2385.25


### Final challenge question: What are the costs paid by subject area?

In [15]:
# For the challenge, I'll assume getting the cost paid by subject area can be reflected by each 'Journal title'
# and summing the cost for the articles associated with each. 
df.groupby('Journal title').sum().sort_values(by='cost', ascending=False)


,cost
Journal title,
plos one,23240.23
elsevier,18377.95
journal of general virology,16275.00
journal of biological chemistry,11581.20
oncogene,9494.45
behavioral neuroscience,9443.12
neuroimage,8769.28
journal of clinical endocrinology and metabolism,8546.43
british journal of pharmacology,8539.39
